In [1]:
from datasets import load_dataset, DatasetDict
from glob import glob
import random
random.seed(42)

all_file_list = glob(pathname="gpt2_data/*/**")
test_file_list = random.sample(all_file_list, 50)
train_file_list = [i for i in all_file_list if i not in test_file_list]

len(train_file_list), len(test_file_list)

(9463, 50)

In [2]:
raw_datasets =load_dataset("csv",data_files={'train':train_file_list,'valid':test_file_list}, cache_dir="cache_data")

raw_datasets

Resolving data files:   0%|          | 0/9463 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Using custom data configuration default-826a877854e349d6


Computing checksums of downloaded files. They can be used for integrity verification. You can disable this by passing ignore_verifications=True to load_dataset


Computing checksums:  19%|#9        | 1828/9513 [00:05<00:21, 365.41it/s]

...

Generating valid split: 0 examples [00:00, ? examples/s]

c:\Users\yuanz\anaconda3\envs\mynet2\lib\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
c:\Users\yuanz\anaconda3\envs\mynet2\lib\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
c:\Users\yuanz\anaconda3\envs\mynet2\lib\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.rea

Dataset csv downloaded and prepared to d:/数据/解压后数据/cache_data/csv/default-826a877854e349d6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'file_path'],
        num_rows: 9230997
    })
    valid: Dataset({
        features: ['content', 'file_path'],
        num_rows: 48228
    })
})

In [3]:
from transformers import AutoTokenizer, AutoConfig

context_length = 512
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

outputs = tokenizer(
    raw_datasets["train"][:2]["content"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 2
Input chunk lengths: [66, 73]
Chunk mapping: [0, 1]


In [4]:
tokenizer.add_special_tokens(special_tokens_dict={'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'})

1

In [5]:
def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

  0%|          | 0/9231 [00:00<?, ?ba/s]

  0%|          | 0/49 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 6129382
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 39353
    })
})

In [6]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [7]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 102.1M parameters


In [8]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [9]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: torch.Size([5, 512])
attention_mask shape: torch.Size([5, 512])
labels shape: torch.Size([5, 512])


In [10]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="chinese_gpt2_big",
    per_device_train_batch_size=20,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    eval_steps=2_000,
    logging_steps=2_000,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=2_000,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)

Using cuda_amp half precision backend


In [11]:
trainer.train()

c:\Users\yuanz\anaconda3\envs\mynet2\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6129382
  Num Epochs = 2
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 160
  Gradient Accumulation steps = 8
  Total optimization steps = 76616
  Number of trainable parameters = 102069504


  0%|          | 0/76616 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 39353
  Batch size = 16


{'loss': 4.023, 'learning_rate': 0.0004997842653586619, 'epoch': 0.05}


  0%|          | 0/2460 [00:00<?, ?it/s]

...

Saving model checkpoint to chinese_gpt2_big\checkpoint-76000
Configuration saved in chinese_gpt2_big\checkpoint-76000\config.json
Configuration saved in chinese_gpt2_big\checkpoint-76000\generation_config.json


{'eval_loss': 2.1710739135742188, 'eval_runtime': 234.8578, 'eval_samples_per_second': 167.561, 'eval_steps_per_second': 10.474, 'epoch': 1.98}


Model weights saved in chinese_gpt2_big\checkpoint-76000\pytorch_model.bin
tokenizer config file saved in chinese_gpt2_big\checkpoint-76000\tokenizer_config.json
Special tokens file saved in chinese_gpt2_big\checkpoint-76000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 224054.7423, 'train_samples_per_second': 54.713, 'train_steps_per_second': 0.342, 'train_loss': 2.451168146737871, 'epoch': 2.0}


TrainOutput(global_step=76616, training_loss=2.451168146737871, metrics={'train_runtime': 224054.7423, 'train_samples_per_second': 54.713, 'train_steps_per_second': 0.342, 'train_loss': 2.451168146737871, 'epoch': 2.0})

In [12]:
# import torch
# from transformers import pipeline

# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# pipe = pipeline(
#     "text-generation", model="codeparrot-ds/checkpoint-2500", device=device
# )

In [13]:
# txt = """\
# # create some data
# x = np.random.randn(100)
# y = np.random.randn(100)

# # create scatter plot with x, y
# """
# print(pipe(txt, num_return_sequences=1)[0]["generated_text"])